# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("./data/the-office-lines - scripts.csv")

In [4]:
print(dataset.shape)
dataset.head()

(59909, 7)


,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [12]:
dataset.isna().sum()

id           0
season       0
episode      0
scene        0
line_text    0
speaker      0
deleted      0
dtype: int64

In [17]:
file = open("Dataset_script.txt", "w")  
for i in range(len(dataset)):
    print("{} {}: {}\n".format(i,dataset.speaker[i],dataset.line_text[i]))
    dialogue = dataset.speaker[i]+": "+dataset.line_text[i]
    file.writelines(dialogue) 
file.close() 

0 Michael: All right Jim. Your quarterlies look very good. How are things at the library?

1 Jim: Oh, I told you. I couldn't close it. So...

2 Michael: So you've come to the master for guidance? Is this what you're saying, grasshopper?

3 Jim: Actually, you called me in here, but yeah.

4 Michael: All right. Well, let me show you how it's done.

5 Michael: [on the phone] Yes, I'd like to speak to your office manager, please. Yes, hello. This is Michael Scott. I am the Regional Manager of Dunder Mifflin Paper Products. Just wanted to talk to you manager-a-manger. [quick cut scene] All right. Done deal. Thank you very much, sir. You're a gentleman and a scholar. Oh, I'm sorry. OK. I'm sorry. My mistake. [hangs up] That was a woman I was talking to, so... She had a very low voice. Probably a smoker, so... [Clears throat] So that's the way it's done.

6 Michael: I've, uh, I've been at Dunder Mifflin for 12 years, the last four as Regional Manager. If you want to come through here... See w

919 Phyllis: Well, uh, for decorations, maybe we could... it's stupid, forget it.

920 Angela: What?

921 Phyllis: I was just going to say, maybe we could have streamers, but that's dumb, everybody has streamers. Never mind.

922 Angela: No, yeah, I think that's a good idea.

923 Phyllis: Yeah?

924 Angela: What color do you guys think?

925 Phyllis: Well, there's green, um, blue... yellow... red...

926 Pam: How about green?

927 Angela: I think green is kind of whoreish.

928 Pam: This was tough. I suggested we flip a coin. But Angela said she doesn't like to gamble. Of course by saying that she was gambling that I wouldn't smack her.

929 Michael: These are my party-planning beeyatches. Pulled off an amazing '80s party last year. Off the hook!

930 Michael: So I was thinking, if you haven't already got a cake, um, maybe going for one of those ice-cream cakes from Baskin-Robbins. Those are very good. Very Delicious.

931 Angela: Meredith's allergic to dairy, so...

932 Michael: She's


1853 Michael: No, it is a perfect smell-alike. I'm not paying for the label. Right here. Give it.

1854 Ryan: Well, it's empty.

1855 Michael: Not it's not, there's some in the straw. [Michael opens bottle and wipes straw along his neck] There, now you may throw it out.

1856 Ryan: Wow. How many filet-o-fishes did you eat?

1857 Michael: That's over several months, Ryan.

1858 Ryan: [Under his breath] Still.

1859 Jim: What's up?

1860 Pam: I'm bored.

1861 Jim: Thank you for choosing me.

1862 Pam: No, I'm kidding. Um, so you got big plans this weekend?

1863 Jim: Ah, well I think I'm gonna see Katy.

1864 Pam: Really?

1865 Jim: Yeah.

1866 Pam: What are you guys going to do?

1867 Jim: Oh, man I don't know. Uh, dinner, drinks, movie, matching tattoos.

1868 Pam: That's great.

1869 Jim: And stuff... yeah.

1870 Pam: That's cool.

1871 Jim: What are you doing?

1872 Pam: I, I was gonna say, I think that um, we're gonna help Roy's cousin move.

1873 Jim: Okay.

1874 Pam: Cause Roy's 


2816 Michael: Home, sweet home.

2817 Dwight: Which one's yours?

2818 Michael: Right there. My sanctuary. My party pad. Someday I can just see my grandkids learning how to walk out here. Hang a swing from this tree. Push them back... wait... [turns around] no, it's this one, right here. Home, sweet home.

2819 Jim: [bounces ball off wall with Toby] So that's what this sound is all day.

2820 Carol: Michael, this is Bill. He's the head of the condo association.

2821 Michael: Oh, how are you? Nice to meet you, Bill. Bill. Mr. Bill. OHHH NOOO. MR. BILL. OHHH! SNL? When they pull him apart? He'd always get rolled over by something.

2822 Bill: Nice to meet you.

2823 Michael: Nice to meet you too.

2824 Dwight: This is smaller than your old place.

2825 Michael: Yeah, small. I'm buying it and I'm not renting it. So, it's still an upgrade. He doesn't know anything about property ownership. Kind of an idiot. Um.

2826 Dwight: Actually, I do own property. My grandfather left me a 60 acre w


3763 Pam: Oh, hey, Jim. Wait, stop. Um, I'm sorry... for pushing you towards Cumberland. Seriously, if you left here, I would blow my brains out.

3764 Jim: [motions for her to follow him] Come on.

3765 Jim: That's just a figure of speech, you know? Blow your brains out? Come on. All it really means is that we're friends. Who else is she gonna talk to if I'm gone, right? I mean, if she left, I wouldn't blow my brains out. Of course, I would take that job in Maryland. Because it's double the pay, and soft shell crab just happens to be my favorite food.

3766 Michael: I  love Halloween. You know, it's just, it's just fun. Every year, it's just fun. Last Halloween I came as Janet Jackson's boob. It was topical. People got a... a big kick out of it. The year before that, I came as Monica Lewinsky, and I wore a stained dress. The year before that, I also came as Monica Lewinsky. And before that, I was O.J. It was pretty funny. Oh, I wish you were here last year.

3767 Children: [ringing t

...


16551 Dwight: I am gonna be your new boss. [laughs] It's my greatest dream come true.  Welcome to the Hotel Hell.  Check in time is now.  Check out time is never.

16552 Jim: Does my room have cable?

16553 Dwight: No. And the sheets are made of fire.

16554 Jim: Can I change rooms?

16555 Dwight: Sorry, we're all booked up. Hell convention in town.

16556 Jim: Can I have a late checkout?

16557 Dwight: I'll have to talk to the manager.

16558 Jim: You're not the manager even in your own fantasy?

16559 Dwight: I'm the owner.  The co-owner. With Satan!

16560 Jim: Okay, just so I understand it, in your wildest fantasy you are in Hell and you are co-running a bed and breakfast with the devil?

16561 Dwight: Yeah, but I haven't told you my salary yet.

16562 Jim: Go.

16563 Dwight: Eighty thousand dollars a year.

16564 Dwight: Once I'm officially Regional Manager my first order of business will be to demote Jim Halpert. So I will need a new number two.  My ideal choice? Jack Bauer. But


17474 Phyllis: That's...

17475 Michael: Okay.

17476 Michael: Maybe she hit me.  You ever think of that?  She hit me?  I don't know whose fault it was.  I wasn't exactly looking at the road.

17477 Pam: Hey, Toby. [walks past him to Kelly's cubicle]

17478 Toby: Hey.

17479 Pam: Kelly, we're going to visit Meredith at lunch.  [Kelly's lying down on floor, head unseen under desk.  She rolls to her side, away from Pam]  Kelly?

17480 Toby: Well, Kelly's been so upset about Ryan dumping her, that she can't even talk. [grins]

17481 Kelly: [from under desk]  I don't want to talk to anybody about Ryan!  Please, go away!

17482 Angela: They don't have this kind of technology at the vet's.  It's not far.

17483 Oscar: It's fair.

17484 Michael: You know what, why don't go around the room and say one memory of Meredith.  I'll start.  I liked her jumpers.  They reminded me of Kindergarten.  Jim?

17485 Kevin: I'll always remember Meredith's back.  That's all I can see from my desk.  All that 


18644 Andy: I don't know yet, I have to call you back.

18645 Voice #1 on phone: You have to give us something...

18646 Andy: I'll call you back.

18647 Angela: I have to go clean up after the party.

18648 Michael: What a horrible day.

18649 Dwight: Blah.

18650 Michael: Bluh.

18651 Dwight: Uhh.

18652 Michael: Well, I need to get the horrible taste of this pizza out of my mouth.  I'd really like some sushi.  I was hoping that I would have New York style sushi today.  And you know what?

18653 Dwight: What?

18654 Michael: I'm going to get it.

18655 Dwight: Coopers has calamari.

18656 Michael: Uh-uh, no, there is only one place where they authentic New York style sushi.

18657 Dwight: Tokyo?

18658 Michael: New York.  Wanna go?

18659 Dwight: Yes.

18660 Michael: Alright, you drive.

18661 Dwight: Okay.

18662 Dwight: Nice.

18663 Michael: Here we go.

18664 Dwight: Woo-hoo.

18665 Michael: Mmm.

18666 Dwight: Yum.

18667 Bartender: I'm sorry, you guys are going to have to leave


19477 Andy: No, no no no, this is good. You know how we haven't really gotten anywhere that I want to get to physically yet? Well, last night, that changed. We're makin' out, I'm kissin' her neck, and cheek and her ear lobe, and she's not really kissing me back. But she closes her eyes and she's like, 'Oh D, oh D.'

19478 Dwight: She called you D?

19479 Andy: Yeah. D for Andy.

19480 Dwight: Oh D.

19481 Andy: Oh D.

19482 Dwight: [whispers] Oh D.

19483 Andy: Oh D!

19484 Both: Ohhhhhh D! [laughing] Ohhhh D!

19485 Michael: I'm about to send the ad to corporate... and it is sent. They'll probably watch it right away. I know I would. Okay. [dials phone]

19486 Pam: [on phone] Yes?

19487 Michael: Pam, please clear my phone lines.

19488 Pam: Certainly. [makes beeping noises] Okay, clear.

19489 Michael: They could call at any second now. [sighs] Oh God...I better call.

19490 Michael: [Ten days later] Well, it's been tough. The uh, geniuses at corporate rejected my commercial and ton

UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 0: character maps to <undefined>

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 30)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 30:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    counts = Counter(text)
    vocab = sorted(counts, key = counts.get ,reverse = True)
    int_to_vocab = {ii: word for ii, word in enumerate(vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    # return tuple
    return (vocab_to_int,int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {
        '.' : '||Period||',
        ',' : '||Comma||',
        '"' : '||Quotation_Mark||',
        ';' : '||Semicolon||',
        '!' : '||Exclamation_Mark||',
        '?' : '||Question_Mark||',
        '(' : '||Left_Parentheses||',
        ')' : '||Right_Parentheses||',
        '-' : '||Dash||',
        '\n': '||Return||'
    }
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [25]:
vocab_to_int, int_to_vocab = create_lookup_tables(text)

In [26]:
len(vocab_to_int)

72

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [24]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    n_batches = len(words)//sequence_length
    words = words[:n_batches*sequence_length]
    train_x = np.zeros([n_batches*sequence_length,sequence_length])
    train_y = np.zeros(n_batches*sequence_length)
    
    for i in range(len(words)):
        if i+sequence_length < len(words):
            train_x[i] = words[i: i+sequence_length]
            train_y[i] = words[i+sequence_length]

    
    train_x = np.array(train_x,np.int64)
    train_y = np.array(train_y,np.int64)
    
    # create Tensor datasets
    data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)    
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [25]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   0,   0,   0,   0],
        [ 34,  35,  36,  37,  38],
        [ 10,  11,  12,  13,  14],
        [  3,   4,   5,   6,   7],
        [ 24,  25,  26,  27,  28],
        [ 19,  20,  21,  22,  23],
        [  0,   1,   2,   3,   4],
        [ 16,  17,  18,  19,  20],
        [ 41,  42,  43,  44,  45],
        [ 33,  34,  35,  36,  37]])

torch.Size([10])
tensor([  0,  39,  15,   8,  29,  24,   5,  21,  46,  38])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [26]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        out = out.view(batch_size, -1, self.output_size)
        # get last batch
        out = out[:, -1]
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [27]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp = inp.cuda()
        target = target.cuda()
    
    hidden = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()

    # get the output from the model
    output, hidden = rnn(inp, hidden)
    
    # perform backpropagation and optimization
    loss = criterion(output.squeeze(), target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(),5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [29]:
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 128
# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [30]:
# Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 400
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 1000

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from workspace_utils import active_session

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 5.029729856014252

Epoch:    1/15    Loss: 4.519282895088196

Epoch:    1/15    Loss: 4.386396594285965

Epoch:    1/15    Loss: 4.309932729959488

Epoch:    1/15    Loss: 4.249298780918122

Epoch:    1/15    Loss: 4.210589300870895

Epoch:    2/15    Loss: 4.100319813204756

Epoch:    2/15    Loss: 4.0083199634552

Epoch:    2/15    Loss: 4.014116450548172

Epoch:    2/15    Loss: 4.00613280582428

Epoch:    2/15    Loss: 4.020351616859436

Epoch:    2/15    Loss: 4.000258942604065

Epoch:    3/15    Loss: 3.917536184350629

Epoch:    3/15    Loss: 3.852521380662918

Epoch:    3/15    Loss: 3.867620030164719

Epoch:    3/15    Loss: 3.8673546783924104

Epoch:    3/15    Loss: 3.8533950176239014

Epoch:    3/15    Loss: 3.880503498792648

Epoch:    4/15    Loss: 3.8151049736249623

Epoch:    4/15    Loss: 3.744961222887039

Epoch:    4/15    Loss: 3.762734231710434

Epoch:    4/15    Loss: 3.779191076517105

Epoch:    4/15    Loss: 3.

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:**<br/>
I took reference from my previous exercises(Skip-gram Word2Vec, Simple RNN, and Sentiment Analysis with an RNN).<br/>
I started with 5 epopchs but the loss decreases significantly when no. of epochs is 15 ( didn't increase further because the loss started increasing).<br/>
I choosed common values for sequence length (10), batch size (128) and learning rate (0.001)<br/>
2-3 is the good number of hidden lstm layers for model, increasing further doesn't help much.<br/>
For hidden dimension I selected a number between 256 - 512.<br/>
Epoch:   15/15    Loss: 3.4608027081489565<br/>





---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [35]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry:

elaine:(to george) hey, what are you gonna do?

elaine: i got a little bit to my father.

elaine:(quietly,) yeah, yeah.

jerry: i think you're gonna go.

jerry: i don't know...

jerry: well, you should have to get the whole thing to get out.

jerry: yeah.

elaine: you know, i think it's a very good idea. i mean, it's like a lot more important.

george:(laughing) oh, you know what i got to do? i mean, i know what it is.

elaine: i think it's a little effeminate.

jerry: oh.

george:(to jerry) you know, you know, if they don't mind, i was thinking about this woman, he was the best, but the guy who gave me a chance.

george: i can't believe you said you were going to the airport.

george: i don't know.

george: well, i was thinking about the other guy.

elaine: oh, no, not, uh, the one.

jerry: well, i don't know what i'm gonna do to the place, you were just trying to get the ball.

jerry: i don't know what to do.

elaine: well, what about your name.

jerry: i thought he said you 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [14]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.